<a href="https://colab.research.google.com/github/Nikolau2321/Kaggle-Spaceship-Titanic/blob/main/NEW_%D0%A4%D0%B8%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from importlib import reload
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras import backend as K
import tensorflow as tf
import glob
from shutil import copyfile
from keras.models import load_model
from keras import optimizers

In [4]:
img_shape = (224, 224, 3)
#загрузка модели
resnet_model = ResNet50(include_top=False, input_shape=img_shape)

94765736/94765736 [==============================] - 1s 0us/step


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import zipfile
zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Дипломный_проект/images_training_rev1.zip', 'r').extractall()

In [7]:
zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Дипломный_проект/training_solutions_rev1.zip', 'r').extractall()

In [8]:

# Загрузка изображений и аугментация


classes = [
    'Class1.1', 'Class1.2', 'Class1.3', 'Class2.1', 'Class2.2', 'Class3.1',
    'Class3.2', 'Class4.1', 'Class4.2', 'Class5.1', 'Class5.2', 'Class5.3',
    'Class5.4', 'Class6.1', 'Class6.2', 'Class7.1', 'Class7.2', 'Class7.3',
    'Class8.1', 'Class8.2', 'Class8.3', 'Class8.4', 'Class8.5', 'Class8.6',
    'Class8.7', 'Class9.1', 'Class9.2', 'Class9.3', 'Class10.1', 'Class10.2',
    'Class10.3', 'Class11.1', 'Class11.2', 'Class11.3', 'Class11.4',
    'Class11.5', 'Class11.6'
]


def append_ext(fn):
    return fn + ".jpg"

traindf = pd.read_csv('/content/training_solutions_rev1.csv')
traindf["id"] = traindf['GalaxyID'].astype(str).apply(append_ext)

datagen = ImageDataGenerator(
    fill_mode='nearest',
    cval=0,
    rescale=1. / 255,
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2) #2% процента

train_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/images_training_rev1/",
    x_col="id",
    y_col=classes,
    subset="training",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="other",
    target_size=(224, 224))

valid_generator = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/images_training_rev1/",
    x_col="id",
    y_col=classes,
    subset="validation",
    batch_size=64,
    seed=123,
    shuffle=True,
    class_mode="other",
    target_size=(224, 224))

STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

Found 49263 validated image filenames.
Found 12315 validated image filenames.


ResNet-50 — сверточная нейронная сеть, обученная на более чем миллионе изображений из базы данных ImageNet. Сеть состоит из 50 слоев и может классифицировать изображения по 1000 категориям объектов, таким как клавиатура, мышь, карандаш и многие животные.


Отрегулируем выходные слои загруженной модели в соответствии с заданными данными обучения.

In [9]:
# Сгладим вывод последнего слоя перед добавлением выходного слоя (плотный слой)
x = Flatten()(resnet_model.output)

# Добавим выходной слой (количество выходов = 37)
x = Dense(len(classes), activation='sigmoid')(x)

# Загружаем модифицированную модель
model = Model(inputs=resnet_model.input, outputs=x)

In [10]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [11]:
#Тонкая настройка всех слоев модели | Адам Оптимизатор 

for layer in model.layers:
    layer.trainable = True

optimizer = keras.optimizers.Adam(lr=0.001, decay=5e-4)

model.compile(optimizer, loss='mse', metrics=["accuracy"])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping


class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))


early_stopping = EarlyStopping(
    monitor='val_loss', patience=4, verbose=1, mode='auto')

history = LossHistory()

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='today_5_epochs/weights.hdf5', verbose=2, save_best_only=True)

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=5,
    callbacks=[history, checkpointer, early_stopping])

<ipython-input-13-2075201587ea>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/5
 53/769 [=>............................] - ETA: 9:44:03 - loss: 0.0484 - accuracy: 0.4148

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(hist.epoch, hist.history['loss'], label='Training Loss')
plt.plot(
    hist.epoch, hist.history['val_loss'], label='Validation', linestyle='--')
plt.xlabel("Epochs")
plt.ylabel("RMSE")
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model

model = load_model('today_5_epochs/weights.hdf5')


Загрузим лучшую модель из сохраненного файла весов

Подгтовим для тестовых данных

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    "/content",
    class_mode=None,
    color_mode="rgb",
    batch_size=1,
    target_size=(224, 224),
    seed=123,
    shuffle=False)

Прогнозы для тестовых данных

In [ ]:
test_generator.reset()

predictions = model.predict_generator(
    test_generator,
    steps=test_generator.n / test_generator.batch_size,
    verbose=1)

print(predictions.shape)

In [ ]:
zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/Дипломный_проект/all_zeros_benchmark.zip', 'r').extractall()

In [ ]:
def get_id(fname):
    return fname.replace(".jpg", "").split("/")[1]


header = open('/content/all_zeros_benchmark.csv', 'r').readlines()[0]

with open('Output.csv', 'w') as outfile:
    outfile.write(header)
    for i in range(len(test_generator.filenames)):
        id_ = (get_id(test_generator.filenames[i]))
        pred = predictions[i]
        outline = id_ + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")